# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "AAPL stock data, statistical modeling, machine learning techniques"}

> Entering new CrewAgentExecutor chain...
I need to continuously monitor and analyze market data for AAPL to identify trends and predict market movements using statistical modeling and machine learning techniques.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL?p=AAPL"} 

Apple Inc. (AAPL) Stock Price, Ne

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced C

Final Answer: 
Apple Inc. (AAPL) is a leading technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products, including the iPhone, Mac, iPad, and various accessories like AirPods and Apple Watch. Additionally, Apple provides services such as AppleCare support, cloud services, the App Store for downloading applications and digital content, and subscription services like Apple Arcade, Apple Fitness+, Apple Music, and Apple TV+.

As of the most recent data, Apple's stock price is $191.06, with a market cap of $2.93 trillion. The company has shown consistent growth over the years, with a 1-year return of 9.49% and a 5-year return of 344.30%. Apple's financial highlights include a profit margin of 26.31%, return on assets of 22.07%, and revenue of $381.62 billion. The company also has a strong balance sheet with total cash of $67.15 billion and levered free cash flow of $84.73

Final Answer: I have gathered insights on AAPL from the Investopedia website. The next step is to analyze the information provided and develop potential trading strategies for day trading based on the user's risk tolerance.

> Finished chain.
 

I have gathered insights on AAPL from the Investopedia website. The next step is to analyze the information provided and develop potential trading strategies for day trading based on the user's risk tolerance.

Final Answer: 
After analyzing the insights gathered on AAPL from various sources, including the Investopedia website, I have developed the following potential trading strategies for AAPL that align with the user's medium risk tolerance and preference for Day Trading:

1. Momentum Trading Strategy: This strategy involves buying AAPL shares when the stock price shows upward momentum and selling when it shows signs of slowing down. It is suitable for short-term traders looking to capitalize on short price movements.

2. Breakout Trading St

Thought: I have gathered information on potential trading strategies for AAPL from the internet search results.

Final Answer: The search results provided insights into various trading strategies for AAPL, such as buying on weakness to certain value levels, using options strategies like the calendar spread, and considering the company's fundamentals and technical characteristics. It is essential to analyze these strategies further to determine the best alignment with the current market conditions, taking into account factors like implied volatility, earnings, and support/resistance levels.

> Finished chain.
 

The search results provided insights into various trading strategies for AAPL, such as buying on weakness to certain value levels, using options strategies like the calendar spread, and considering the company's fundamentals and technical characteristics. It is essential to analyze these strategies further to determine the best alignment with the current market conditions, takin

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Based on the analysis of the current market conditions for AAPL and the proposed Breakout Trading Strategy, the Risk Advisor has conducted a comprehensive risk analysis and provided detailed mitigation recommendations.

The high volatility in the market conditions for AAPL, driven by upcoming earnings reports and potential market events, presents several risks for the Breakout Trading Strategy. These risks include potential false breakouts, sudden reversals in stock price momentum, and increased market sensitivity to external events such as earnings reports and industry trends.

To mitigate these risks, the Risk Advisor recommends implementing safeguards such as setting stop-loss orders to limit potential losses, closely monitoring key resistance levels and market sentiment indicators, and diversifying the trading portfolio to spread risk exposure. It is crucial to carefully assess the risk-reward ratio of the Breakout Trading Strategy for AAPL and adjust trading positions accordingly to align with the firm's risk tolerance and overall investment goals.

Overall, the Risk Advisor's analysis provides valuable insights into the potential risks associated with the proposed trading strategies for AAPL and offers effective mitigation strategies to ensure the success of the team in navigating the volatile market conditions.